This notebook is about:
- Interpolation/Extrapolation of the signal (resample)
- Circular Buffer Shift
- Analyze Spectral Content of the Signal
- Features Extraction
- Model of Lung from a Breath of 3sec Instance
- Finding the input Uin Delay
- Summary

Introduction:

Three ways to analyze the spectral contents of the non-uniform(native) time step signal.
- Non uniform fft (nfft), this is a special fft function
- Lomb-Scargle Periodogram
- Resample the signals, then take uniform fft

The nfft can perform fft based on the non uniform time step. After nfft, all the time steps are in uniform.  The Lung transfer can be calculated based on the uniform time-step. After the Lung model, Lung output, and Uin delay are calculated, then the Lung output in the time-domain can be transformed back to the native non uniform time-step. 

Resample is another way to transform non-uniform to uniform time-steps. After resample, you can either use scipy.fft or nfft to transform to freq-domain, then the Lung model can be derived in the same way as described above.

I don't use Lomb-Scargle to derive the Lung transfer function.

The difference between nfft and resample method is that nfft has uniform time-step for a given breath while resample is for all breaths.
Suppose you have 3 step-sizes, small, medium, and large. Native time-step means that there can be any combinations in the 80 steps. nfft will have a small of this breath-id, and medium for other breath-id, etc. Resample will have standard, say medium for all breath-ids. Everytime you change the time step, the delay, filter coeff, or PID loop filter need to be recalculated. So, what is the advantage? The advantage is that, the resample method lets the NN to focus just on the Lung Model rather than combination recalculation of filter coeff, etc.


In [ ]:
#https://github.com/jakevdp/nfft#basic-usage
!pip install nfft

from nfft import nfft         
from nfft import nfft_adjoint 

In [ ]:
import os
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
#import tensorflow as tf

import scipy as sp
from scipy import signal
from scipy.signal import butter,filtfilt,freqz
import scipy.ndimage
import scipy.fftpack
from scipy.fft import rfft, irfft
import scipy.signal as signal
from scipy.interpolate import interp1d

In [ ]:
df_train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df_test  = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
df_sub   = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
df_train.head(3)

total id = 6,036,000 = breath_id * time-step = 75450 * 80

fs = 80 uniform samples per 3 sec = 80/3 = 26.67 Hz ~ 27Hz

In [ ]:
lag=1
shift_t_step = df_train.groupby('breath_id')['time_step'].shift(lag).fillna(0)
df_train[f't_d{lag}'] = df_train['time_step'] - shift_t_step

df_train['t_max']  = df_train.groupby('breath_id')[f't_d{lag}'].transform('max')    #
df_train['t_mean']  = df_train.groupby('breath_id')[f't_d{lag}'].transform('mean')  #
df_train['t_std']  = df_train.groupby('breath_id')[f't_d{lag}'].transform('std')    #
df_train['t_uniform']  = df_train['t_std']/df_train['t_mean']                       #lower the better

In [ ]:
breath_id_1 = df_train.query('breath_id == 1').reset_index(drop = True)
#breath_id1 = df_train.query('breath_id == 44245').reset_index(drop = True)
breath_id_1

In [ ]:
col =['time_step','u_in','u_out','pressure']
col2 =['time_step','t_d1','t_mean','t_std','t_uniform']
breath_id_1[col].iloc[:80].plot()         
breath_id_1[col2].iloc[:80].plot()

In [ ]:
breath_id_12 = df_train.query('breath_id == 12').reset_index(drop = True)
#breath_id_12
breath_id_12[col].iloc[:80].plot()
breath_id_12[col2].iloc[:80].plot()

In [ ]:
df_test.head(3)

In [ ]:
#define native fs the mean non uniform time-step interval
dt1 = breath_id_1['t_mean'][0]
print(f'native time-interval=',dt1,',max acq time=',80*dt1, ',native fs=', 1/dt1, 'Hz')

In [ ]:
dt12 = breath_id_12['t_mean'][0]
print(f'native time-interval=',dt12,',max acq time=',80*dt12 , ',native fs=', 1/dt12, 'Hz')

In [ ]:
def circular_shift_forward(arr, nroll):
    arr_roll = np.zeros((nroll,len(arr))) #expand the arr dimension
    #print(np.shape(arr_roll),nroll)
    for i in range(nroll):
        #print(i, np.roll(arr,i))
        arr_roll[i] = np.roll(arr,i)
    return arr_roll

def circular_shift_backward(arr, nroll):
    arr_roll = np.zeros(np.shape(arr))  #keep dimension
    for i in range(nroll):
        #print(i, np.roll(arr[i],-i))
        arr_roll[i] = np.roll(arr[i],-i)
    return arr_roll

#interpolation has a "natural" smoothing action and tend to represent lower freq
#In this application, it's a low freq.
def interp1d_to_54Hz(t_54Hz, t_native, y_native):
    #these are the number of samples cannot be interpolated beyond native time-steps
    idx_max = np.where(t_54Hz > t_native[-1])[0]
    
    y_interp = interp1d(t_native, y_native)
    dat_interp = y_interp(t_54Hz[0:idx_max[0]])
    print(len(dat_interp))
    
    #Extrapolation, just fill the tail with their last value. 
    #One can fill the tail by extrapolating from the last to the first value.
    dat_interp = np.append(dat_interp, y_native[-1]*np.ones(len(idx_max))) 
    return dat_interp

def interp1d_reverse(t_native, t_27Hz, y_27Hz):
    y_interp = interp1d(t_27Hz, y_27Hz)
    dat_interp = y_interp(t_native)
    #print(len(dat_interp))
    return dat_interp

def mae_loss(y_true, y_pred):
    mae = np.sum(np.abs(y_true - y_pred))
    return mae / len(y_true)

def plot_arr(arr):
    for i in range(len(arr)):
        plt.plot(arr[i])
        
def uniform_fft(arr):
    fft = np.fft.fft(arr)
    psd = np.abs(fft) ** 2
    fftfreq = sp.fftpack.fftfreq(len(psd),1/fs)
    i = fftfreq > 0
    return fftfreq,i, psd       

In [ ]:
#Resample signal
#Insert zeros to upsample the signal
#Apply an FIR antialiasing filter
#Discard samples to downsample

#transform all non-uniform native fs to uniform fs @54Hz by interpolation/extrapolation
#- higher freq give better interpolation
#- move signal away from Nyquist, then applied antialiasing filter
#then downsample to uniform 27Hz

#splitted 3sec into uniform 160 steps, this will double the sampling rate = 2*27Hz
#fs = 54Hz, uniform time-step setting 
𝛿t_res_54Hz = 3/(2*80)                  #each time-step is 𝛿t sec.
fs_54Hz = 1/𝛿t_res_54Hz                 #sample freq is 53.33 Hz
len_54Hz = int(3*fs_54Hz)               #len of 3sec samples
𝛿f_res_54Hz = fs_54Hz/len_54Hz          #freq step @54Hz
𝛿f_axis_54Hz = 𝛿f_res_54Hz*np.arange(0, len_54Hz, dtype=float)
𝛿t_axis_54Hz = 𝛿t_res_54Hz*np.arange(0, len_54Hz, dtype=float) #time-step x-axis

#decimate down to 27Hz, uniform time-step
fs_27Hz = np.ceil(fs_54Hz/2)
𝛿t_res_27Hz = 2*𝛿t_res_54Hz
𝛿f_res_27Hz = 2*𝛿f_res_54Hz #freq step @54Hz
𝛿f_axis_27Hz = 𝛿f_axis_54Hz[0:-1:2]/2     #freq-step x-axis @27Hz, 80points, DC, pos, Nyq, neq
𝛿t_axis_27Hz = 𝛿t_axis_54Hz[0:-1:2]       #time-step x-axis @27Hz
len_27Hz = int(len_54Hz/2)
#len_54Hz, 𝛿t_res_54Hz, 𝛿f_res_54Hz 
print( f'uniform sample rate=',fs_27Hz,'Hz,len sample=',len_27Hz, ',time-res=', 𝛿t_res_27Hz, ',freq-res=',𝛿f_res_27Hz)

In [ ]:
T_id_1    = breath_id_1['time_step'].values
P_id_1    = breath_id_1['pressure'].values
Uin_id_1  = breath_id_1['u_in'].values
Uout_id_1 = breath_id_1['u_out'].values

T_id_12    = breath_id_12['time_step'].values
P_id_12    =breath_id_12['pressure'].values
Uin_id_12  =breath_id_12['u_in'].values
Uout_id_12 = breath_id_12['u_out'].values

plt.figure(figsize=(22,4))
plt.subplot(1,4,1)
plt.plot(P_id_1)
plt.plot(P_id_12)
plt.grid()
plt.legend(['press 1','press 12'])
plt.xlabel('time-step (delay-taps)')

plt.subplot(1,4,2)
plt.plot(T_id_1,P_id_1 )
plt.plot(T_id_12,P_id_12)
plt.grid()
plt.legend(['press 1','press 12'])
plt.xlabel('time-step (sec)')

plt.subplot(1,4,3)
plt.plot(T_id_1)
plt.plot(T_id_12)
plt.plot( 𝛿t_axis_27Hz )
plt.plot(Uout_id_1)
plt.plot(Uout_id_12)
plt.grid()
plt.legend(['T_id_1','T_id_12','uniform T-step @27Hz','Uout_id_1','Uout_id_12'])
plt.xlabel('time-step (delay-taps)')

In [ ]:
print(f'id_1 native fs=', 1./np.mean(breath_id_1['t_mean'][0]),', id_12 native fs=', 1./np.mean(breath_id_12['t_mean'][0]))

breath_id_12 fs is faster, at time-step 80, it terminated acquisition at 2.5sec

In [ ]:
trn_step_min = df_train.groupby('breath_id')['time_step'].transform('last').min()
trn_step_max = df_train.groupby('breath_id')['time_step'].transform('last').max()
tst_step_min = df_test.groupby('breath_id')['time_step'].transform('last').min()
tst_step_max = df_test.groupby('breath_id')['time_step'].transform('last').max()
dataset_step_min = min(trn_step_min, tst_step_min)

#trn_step_min, trn_step_max, tst_step_min, tst_step_max, dataset_step_min
print(f'the fastest termination in time =',dataset_step_min, 'sec')

# Interpolation/Extrapolation

- Resample the non-uniform to uniform time-step
- Upsample to 54Hz with 160 uniform time-step 
- Apply interpolation/extrapolation
- then downsample to 27Hz with 80 uniform time-step

In [ ]:
#interp non-uniform to uniform time-step at 54Hz (160 delay taps)

# Finding the interpolation, ID_1, Pressure
pressure_interp = interp1d_to_54Hz(𝛿t_axis_54Hz,T_id_1,P_id_1)
len(pressure_interp) #interpolate 144 samples + 16 samples need extrapolation

In [ ]:
# Finding the interpolation, ID_12, Pressure
pressure_interp2 = interp1d_to_54Hz(𝛿t_axis_54Hz,T_id_12,P_id_12)
len(pressure_interp2) #interpolate 135 samples + 25 samples need extrapolation

In [ ]:
plt.figure(figsize=(18,4))
plt.subplot(1,2,1)

#down sample to 27Hz, need anti-aliasing before down-sample
plt.plot(pressure_interp[:-1:2])  #ignore anti-aliasing for now
plt.plot(pressure_interp2[:-1:2]) #ignore anti-aliasing for now
plt.grid()
plt.legend(['P_interp_id_1','P_interp_id_12'])
plt.xlabel('time-step (delay-taps)')

plt.subplot(1,2,2)
plt.plot(T_id_1,P_id_1)
plt.plot(𝛿t_axis_27Hz, pressure_interp[:-1:2]) #ignore anti-aliasing for now
plt.plot(T_id_12,P_id_12)
plt.plot(𝛿t_axis_27Hz, pressure_interp2[:-1:2]) #ignore anti-aliasing for now
plt.grid()
plt.legend(['P_id_1','P_interp_id_1','P_id_12','P_interp_id_12'])
plt.xlabel('time-step (sec)')

In [ ]:
#extrapolated non uniform native fs to uniform 27Hz
#when interpolate from uniform 27Hz back to native fs, the extrapolated portion will be discarded anyway.
print(f' the tails are extrapolated from' ,T_id_12[-1],'to',  𝛿t_axis_27Hz[-1], 'sec')

In [ ]:
#interpolation error
err_y2 = np.sum(np.abs(P_id_12 - pressure_interp2[:-1:2] ))/len(P_id_12) 
err_y2

#This error is invalid because:
#- data is extrapolated beyond T_id_12[-1] = 2.52 sec
#- data is not aligned, one in native fs and other in uniform fs=27Hz
#For a fair comparison, one need to interp back to native fs

In [ ]:
# Finding the interpolation
# Do the same for u_in, u_out

Uin_interp = interp1d_to_54Hz(𝛿t_axis_54Hz,T_id_1,Uin_id_1)
Uout_interp = interp1d_to_54Hz(𝛿t_axis_54Hz,T_id_1,Uout_id_1)
Uin2_interp = interp1d_to_54Hz(𝛿t_axis_54Hz,T_id_12,Uin_id_12)
Uout2_interp = interp1d_to_54Hz(𝛿t_axis_54Hz,T_id_12,Uout_id_12)

#Extrapolation, just fill the tail with their last value. 
#One can fill the tail by extrapolating from the last to the first value.

In [ ]:
plt.figure(figsize=(18,4))
plt.subplot(131)
plt.plot(Uin_id_1)
plt.plot(Uin_interp[::2])
plt.plot(Uin_id_12)
plt.plot(Uin2_interp[::2])
plt.grid()
plt.legend(['Uin_id_1','Uin_interp_id_1','Uin_id_12','Uin_interp_id_12'])
plt.xlabel('time-step (delay-taps)')

plt.subplot(132)
#plt.plot(T_id_1)
#plt.plot(T_id_12)
#plt.plot( 𝛿t_axis_27Hz )

plt.plot(Uout_id_1,'b')
plt.plot(Uout_interp[::2],'r--')
plt.plot(Uout_id_12,'g')
plt.plot(Uout2_interp[::2],'k.')
plt.grid()
plt.legend(['Uout_id_1','Uout_interp_id_1','Uout_id_12','Uout_interp_id_12'])
plt.xlabel('time-step (delay-taps)')

plt.subplot(133)
plt.plot(T_id_1,Uin_id_1)
plt.plot(𝛿t_axis_27Hz,Uin_interp[::2])
plt.plot(T_id_12,Uin_id_12)
plt.plot(𝛿t_axis_27Hz,Uin2_interp[::2])
plt.grid()
plt.legend(['Uin_id_1','Uin_interp_id_1','Uin_id_12','Uin_interp_id_12'])
plt.xlabel('time-step (sec)')

# Circular Buffer Shift

**The idea** 
- To expand the dimension to 2D, treat this as an image if you want.
- Averaging to reduce noise
- Applicable to target, the NN output as dense expanded dim size, can be different from input dimension.
- If the target is in a non-uniform time-step, the NN will map the uniform input time-step to whatever the target time-step, hopefully.
- The cost function will need to be modified accordingly.

In [ ]:
Uin_27Hz = Uin_interp[0:-1:2]  #ignore anti-aliasing filter
dfs2 = circular_shift_forward(Uin_27Hz, 16)  #expand dimension to 2D

#add noise
img = dfs2 
mean = 0.0   
std = 1.0   
noisy_img = img + np.random.normal(mean, std, img.shape)
noisy_img_clipped = np.clip(noisy_img, 0, 255) 

#unroll
dfs3 = circular_shift_backward(noisy_img_clipped, 16)

#average them, for every 2^n = 16 averaging, you reduced the noise by 3*n dB ~= 12 dB
dfs3_mean = np.mean(dfs3,axis=0)

#Interpolate back to native fs, non-uniform time-step, clean vs. added noise
Uin_reconstruct = interp1d_reverse(T_id_1, 𝛿t_axis_27Hz, dfs3_mean ) #has reconstruct err + noise

#Interpolate back to native fs, non-uniform time-step, clean vs. clean
Uin_recon_clean = interp1d_reverse(T_id_1, 𝛿t_axis_27Hz, Uin_27Hz )  #has pure reconstruct err

In [ ]:
plt.figure(figsize=(22,4))

plt.subplot(151)
plot_arr(dfs2)
plt.title('roll circular shift')

plt.subplot(152)
plot_arr(noisy_img_clipped)
plt.title('add noise')

plt.subplot(153)
plot_arr(dfs3)
plt.title('unroll circular shift')

plt.subplot(154)
plt.plot((dfs2[0]))
plt.plot(dfs3_mean)
plt.title('average')

#plt.figure(figsize=(12,4))
plt.subplot(155)
plt.plot(T_id_1)
plt.plot(Uin_id_1)
plt.plot(Uin_reconstruct)
plt.plot(Uin_recon_clean)
plt.legend(['T_id_1','Uin_id_1','Uin_recon_noise','Uin_recon_clean'])
plt.title('non-uniform time-step')

In [ ]:
print(f'uniform MAE=',mae_loss( dfs2[0], dfs3_mean),', non-unif noise MAE=', mae_loss(Uin_id_1, Uin_reconstruct),', recon MAE=', mae_loss(Uin_id_1, Uin_recon_clean))

In [ ]:
fs = 54      # sample rate, 54 Hz
nyq = 0.5 * fs # Nyquist Frequency

fs_native = np.ceil(1/dt1)
nyq_native = 0.5 *fs_native

order_lpf = 10   #filter order  
lpf_cutoff= nyq-1  #Hz, 

def butter_LPF_filter(data, cutoff_low, fs, order):
    normal_cutoff_low = cutoff_low/nyq
     
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff_low, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

bl, al = butter(order_lpf, lpf_cutoff/nyq, btype='low', analog=False)
wl,hl = freqz(bl,al, fs=fs)

#lpf
plt.figure(figsize=(22,4))
plt.subplot(121)
plt.plot(wl, 20 * np.log10(abs(hl)), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [Hz]')
plt.grid()
plt.title('LPF, cutoff @ 27Hz')

bins=50
plt.subplot(122)
plt.plot(wl[-bins:], 20 * np.log10(abs(hl[-bins:])), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [Hz]')
plt.grid()
plt.title('Zoom-In LPF')

# Spectral content of the signal
- Lomb-Scargle Periodogram
- to analyze spectral content of non-uniform FFT(nFFT) at native sampling rate
- to compare uniform FFT after interpolation at fs=27Hz
- to analyze spectral content during inhale(1st sec) and exhale(3rd sec)
- to analyze phase delay during inhale(1st sec) and exhale(3rd sec)

In [ ]:
#Comparing Uin with vs without anti-aliasing filter

Uin_54Hz = butter_LPF_filter(Uin_interp, lpf_cutoff, fs, order_lpf)  #apply anti-aliasing filter
Uin_27Hz_LPF = Uin_54Hz[::2]    #decimate after anti-aliasing filter
Uin_27Hz     = Uin_interp[::2]  #decimate without anti-aliasing filter

uin_27Hz_norm     = (Uin_27Hz - Uin_27Hz.mean())/Uin_27Hz.std()
uin_27Hz_norm_lpf = (Uin_27Hz_LPF - Uin_27Hz_LPF.mean())/Uin_27Hz_LPF.std()


plt.figure(figsize=(22,4))
for batch in range(1):
    fft_f1, i1, psd1 = uniform_fft(uin_27Hz_norm)
    fft_f2, i2, psd2 = uniform_fft(uin_27Hz_norm_lpf)
    
    plt.subplot(121)
    plt.plot(uin_27Hz_norm,'b')
    plt.plot(uin_27Hz_norm_lpf,'r--')
    plt.ylabel('Amplitude [dB]', color='b')
    plt.xlabel('time-samples')
    plt.grid()
    plt.title('LPF, cutoff @ 27Hz')
    plt.legend(['unfilter','lpf'])
    
    plt.subplot(122)
    plt.plot(fft_f1[i1], 10 * np.log10(psd1[i1]),'b')
    plt.plot(fft_f2[i2], 10 * np.log10(psd2[i2]),'r--')
    plt.xlabel('freq (Hz)')
    plt.grid()
    plt.legend(['unfilter','lpf'])

Comparing Uin with vs without anti-aliasing filter

In [ ]:
#Lomb-Scargle Periodogram for non uniform fs
#Non uniform original Uin
x= T_id_1
y= Uin_id_1
#x= ut
#y= uin
f= np.linspace(0.01, 14, len(x)) 
pgram = signal.lombscargle(x, y, f, normalize=True)
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(x, 10*np.log10(np.abs(y)+1), 'b+-')
plt.xlabel('time (sec)')
plt.grid()
plt.subplot(122)
plt.plot(f, pgram)
plt.xlabel('freq (Hz)')
plt.grid()
plt.show()

Spectral content of the signal using Lomb-Scargel Periodogram

In [ ]:
#df_bid =[56152, 79152, 70251, 69150] # 85113, 49703, 11845, 19375, 69150]
df_bid = list(df_train['breath_id'].sample(4))

N = 80
#x = np.linspace(0.0,0.5, N)
ut_pt5 = np.zeros(N)
𝛿t_27Hz_pt5 = np.zeros(N)
for j in range(N):
    #ut_pt5[j] = 0.5*ut[j]/ut[-1]
    𝛿t_27Hz_pt5[j] = 0.5*𝛿t_axis_27Hz[j]/𝛿t_axis_27Hz[-1]  #norm to 0.5  

plt.figure(figsize=(24,18))

for i in range(0,len(df_bid)):
    #print(df_bid[i])
    uin_1st_sec  = np.zeros(N)
    uin_3rd_sec  = np.zeros(N)
    p_1st_sec    = np.zeros(N)
    p_3rd_sec    = np.zeros(N)
    
    ut       = df_train.loc[df_train['breath_id'] == df_bid[i]]['time_step'].tolist()
    dt_mean  = df_train.loc[df_train['breath_id'] == df_bid[i]]['t_mean'].tolist()[0]
    uin      = df_train.loc[df_train['breath_id'] == df_bid[i]]['u_in'].tolist()
    p        = df_train.loc[df_train['breath_id'] == df_bid[i]]['pressure'].tolist()
    
    R        = df_train.loc[df_train['breath_id'] == df_bid[i]]['R'].unique()[0]
    C        = df_train.loc[df_train['breath_id'] == df_bid[i]]['C'].unique()[0]
    S        = df_train.loc[df_train['breath_id'] == df_bid[i]]['t_max'].unique()[0]
    name     = f'bid : {df_bid[i]}, R:{R}, C:{C}, S:{np.round(S,3)}'
    
    for j in range(N):
        ut_pt5[j] = 0.5*ut[j]/ut[-1]  #norm to 0.5    
    
    #Non-uniform FFT at native fs
    uin_dc = np.mean(uin)
    p_dc = np.mean(p)
    uin_norm = (uin - uin_dc)/np.std(uin)   
    p_norm = (p - p_dc)/np.std(p)
    
    ###############################################
    # taking FFT straight from non-uniform sample
    ###############################################
    native_fs = np.ceil(1/dt_mean)   #native fs
    #print(dt_mean, native_fs)
    f_uin_norm_k = nfft_adjoint(ut_pt5, uin_norm, N, truncated=False)
    f_p_k        = nfft_adjoint(ut_pt5, p_norm, N, truncated=False)
    uf = np.abs(np.fft.ifftshift(f_uin_norm_k)[:int(N/2)+1])
    pf = np.abs(np.fft.ifftshift(f_p_k)[:int(N/2)+1])
    #uniform after nFFT
    
    ###############################################
    # taking FFT on uniform sample after interp and down-sample
    ###############################################
    #interplolated, Uniform Uin and Uniform P
    Uin_interp = interp1d_to_54Hz(𝛿t_axis_54Hz,ut,uin) 
    P_interp   = interp1d_to_54Hz(𝛿t_axis_54Hz,ut,p) 
    #LPF anti-aliasing filter
    Uin_interp_fltr_54Hz = butter_LPF_filter(Uin_interp, lpf_cutoff, fs, order_lpf)
    #Uin_interp_fltr_54Hz = Uin_interp  #skip

    P_interp_fltr_54Hz =  butter_LPF_filter(P_interp, lpf_cutoff, fs, order_lpf)
    #P_interp_fltr_54Hz   = P_interp    #skip 
   
    #Decimate to 27Hz
    Uin_interp_fltr_27Hz = Uin_interp_fltr_54Hz[::2]
    P_interp_fltr_27Hz   = P_interp_fltr_54Hz[::2]
    #Norm
    Uin_interp_fltr_27Hz_dc = np.mean(Uin_interp_fltr_27Hz)
    P_interp_fltr_27Hz_dc = np.mean(P_interp_fltr_27Hz)
    
    uin_norm_27Hz = (Uin_interp_fltr_27Hz - Uin_interp_fltr_27Hz_dc)/np.std(Uin_interp_fltr_27Hz)   
    p_norm_27Hz   = (P_interp_fltr_27Hz - P_interp_fltr_27Hz_dc)/np.std(P_interp_fltr_27Hz) 
    #FFT, uniform after resample, could have use uniform FFT
    f_uin_norm_27Hz_k = nfft_adjoint(𝛿t_27Hz_pt5, uin_norm_27Hz, N, truncated=False) 
    f_p_27Hz_k        = nfft_adjoint(𝛿t_27Hz_pt5, p_norm_27Hz, N, truncated=False) 
    uin_norm_27Hz_f   = np.fft.ifftshift(f_uin_norm_27Hz_k)[:int(N/2)+1]
    p_norm_27Hz_f     = np.fft.ifftshift(f_p_27Hz_k)[:int(N/2)+1]
    
    #DC and intrp_DC should be same, difference is likely due to extrapolation
    print(f'u_dc=',uin_dc,',p_dc=',p_dc,',u_intrp_dc=',Uin_interp_fltr_27Hz_dc,',p_intrp_dc=',P_interp_fltr_27Hz_dc)
    
    ###############################################
    # analyze spectral content on first sec and third sec
    ###############################################
    #copy 15 samples from the first sec and the third sec for FFT
    # avoid transition transient signal during 2nd second
    # avoid initial ramp input offset 5 samples 
    # tone3 or tone9 period is an integer multiple of fs=27Hz, important property to analyze phase delay
    # Two seconds elapsed = 2/3 * 80 = 53 samples
    # Two seconds elapsed have integer 18 cycles(tone9) or 6 cycles(tone3) elapsed with the same starting phase. 
    offset = 5  #avoid initial ramp up input
    elap = 53   #2 sec elapsed
    dur = 15    # 15 samples for FFT
    two_sec = offset+elap
    two_dur = two_sec+dur
    
    uin_1st_sec[:dur]  = uin_norm_27Hz[offset:offset+dur]  #1st sec idx[0:80*1/3]
    uin_3rd_sec[:dur] = uin_norm_27Hz[two_sec:two_dur]  #3rd sec idx[80*2/3:80]
    p_1st_sec[:dur]    = p_norm_27Hz[offset:offset+dur]
    p_3rd_sec[:dur]   = p_norm_27Hz[two_sec:two_dur]    
    #FFT
    uin_1st_sec_k   = nfft_adjoint(𝛿t_27Hz_pt5, uin_1st_sec- np.mean(uin_1st_sec), N, truncated=False) #mean value is DC, important feature
    uin_3rd_sec_k   = nfft_adjoint(𝛿t_27Hz_pt5, uin_3rd_sec- np.mean(uin_3rd_sec), N, truncated=False) #mean value is DC, important feature
    p_1st_sec_k     = nfft_adjoint(𝛿t_27Hz_pt5, p_1st_sec- np.mean(p_1st_sec), N, truncated=False) #mean value is DC, important feature
    p_3rd_sec_k     = nfft_adjoint(𝛿t_27Hz_pt5, p_3rd_sec- np.mean(p_3rd_sec), N, truncated=False) #mean value is DC, important feature

    uin_1st_sec_f   = np.fft.ifftshift(uin_1st_sec_k)[:int(N/2)+1]
    uin_3rd_sec_f   = np.fft.ifftshift(uin_3rd_sec_k)[:int(N/2)+1]
    p_1st_sec_f     = np.fft.ifftshift(p_1st_sec_k)[:int(N/2)+1]
    p_3rd_sec_f     = np.fft.ifftshift(p_3rd_sec_k)[:int(N/2)+1]    
    
    ###############################################
    
    plt.subplot(4,4,4*i+1)
    plt.plot(ut, uin_norm, color='blue')
    plt.plot(ut, p_norm, 'r-')
    plt.grid()
    plt.legend(['u_in','pressure'])
    
    #xf = np.fft.fftfreq(N,1./N) #[0,1,...(N-1),-N,-(N-1),...-1], index 0 is DC, index N/2 is Nyq
    #f_res = 1/dt_mean/N
    
    plt.subplot(4,4,4*i+2)
    plt.plot(2*native_fs*(ut_pt5)[:int(N/2)+1], uf,'b',label='u_in') 
    plt.plot(2*native_fs*(ut_pt5)[:int(N/2)+1], pf,'r--',label='pressure')
    plt.grid()
    plt.title(name)
    plt.legend(['u_in','pressure'])    
    
    plt.subplot(4,4,4*i+3)
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(uin_norm_27Hz_f), 'b', label='u_in') 
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(p_norm_27Hz_f), 'r--', label='pressure')  
    plt.grid()
    plt.legend(['u_in','pressure'])

    plt.subplot(4,4,4*i+4)
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(uin_1st_sec_f), 'b', label='uin_1sec') 
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(uin_3rd_sec_f), 'b+', label='uin_3sec')      
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(p_1st_sec_f), 'r', label='p_1sec') 
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(p_3rd_sec_f), 'r+', label='p_3sec')      
    plt.grid()
    plt.legend(['uin_1sec','uin_3sec','p_1sec','p_3sec'])

The second column plot is the non-uniform FFT at native fs.
- Basically, the low-freq band is close to DC around 1Hz
- Low-freq band magnitude, sometimes high or low. In closed-loop PID, the steady state error depends on the proportional DC gain. High gain, small steady state error.

The third column plot is the uniform FFT at fs=27Hz
- The low-freq band has some variation bandwidth.
- The freq responses of u_in and pressure are used to derive the Lung transfer functions.  Err = Pressure - Lung .* u_in
- If Err = 0, the optimal Lung transfer function Lung_f = [ Pressure .* conj(u_in) ] ./ [ u_in .* conj(u_in) ]
- Need a software package to convert Lung_f to Lung_z(Z), where Z is a delay operator in discrete-time domain. 
- The closed-loop transfer function is defined as H_CL(Z) = Pressure(Z) / Lung_output(Z) => ZEROS / POLES
- The closed-loop dynamic is defined as the denominator polynomial of H_CL(Z)
- The roots(POLES) of the denominator H_CL polynomial must lied inside a unit circle
- The proportional Gain controls the POLES locations, if the POLES are complex(overdamped), real (underdamped), on the unit circle (critical damped).
- What if, in freq-domain, Lung_f is derived from pressure(native fs) vs. u_in(fs=27Hz)? LUNG_f output is in native fs, no need to re-interpolate back to native?

The last column plot is the 1st and 3rd sec of the uniform FFT at fs=27Hz
- The magnitude, bandwidth, and phase delay are explained in next section

In [ ]:
#𝛿f_axis_27Hz                               # 80points, DC, pos, Nyq, neq
𝛿f_x_27Hz = 54*(𝛿t_27Hz_pt5)[:int(N/2)+1]   # 41points, DC, pos, Nyq

#tone8 = 8.   
#tone9 = 9.   
#tone10 = 10. 
#tone8_idx = np.where(np.round(𝛿f_axis_27Hz[:int(N/2)],5) == tone8)
#tone9_idx = np.where(np.round(𝛿f_axis_27Hz[:int(N/2)],5) == tone9)
#tone10_idx = np.where(np.round(𝛿f_axis_27Hz[:int(N/2)],5) == tone10)

tone2_idx = 6    #2Hz / 0.3375 = bin 6
tone3_idx = 9    #3Hz / 0.3375 = bin 9
tone4_idx = 12   #4Hz / 0.3375 = bin 12
tone8_idx = 24   #8Hz / 0.3375 = bin 24
tone9_idx = 27   #9Hz / 0.3375 = bin 27
tone10_idx = 30  #10Hz/ 0.3375 = bin 30

TEST_TONE = 3

if TEST_TONE == 3:
    test_tone_minus1  = tone2_idx
    test_tone         = tone3_idx
    test_tone_plus1   = tone4_idx
else:
    test_tone_minus1  = tone8_idx
    test_tone         = tone9_idx
    test_tone_plus1   = tone10_idx

u_1 = uin_1st_sec_f[test_tone] .round(2)
u_3 = uin_3rd_sec_f[test_tone] .round(2)
deg_u_1 = np.degrees(np.arctan2(u_1.imag,u_1.real)).round(2)
deg_u_3 = np.degrees(np.arctan2(u_3.imag,u_3.real)).round(2)
p_1 = p_1st_sec_f[test_tone].round(2)
p_3 = p_3rd_sec_f[test_tone].round(2)
deg_p_1 = np.degrees(np.arctan2(p_1.imag,p_1.real)).round(2)
deg_p_3 = np.degrees(np.arctan2(p_3.imag,p_3.real)).round(2)
print(f'test tone phase angle in cartesian coordinate (degree)')
print(f'1st uin',u_1,'(',deg_u_1,'deg)')
print(f'3rd uin',u_3,'(',deg_u_3,'deg)')
print(f'1st p',p_1,'(',deg_p_1,'deg)')
print(f'3rd p', p_3,'(',deg_p_3,'deg)')

plt.figure(figsize=(22,4))
plt.subplot(141)   #uin magnitude plot
plt.plot(𝛿f_x_27Hz[test_tone_minus1],np.abs(uin_1st_sec_f[test_tone_minus1]),'b*', label='tone2_1st')
plt.plot(𝛿f_x_27Hz[test_tone],np.abs(uin_1st_sec_f[test_tone]),'r*', label='tone3_1st')
plt.plot(𝛿f_x_27Hz[test_tone_plus1],np.abs(uin_1st_sec_f[test_tone_plus1]),'g*', label='tone4_1st')
plt.plot(𝛿f_x_27Hz[test_tone_minus1],np.abs(uin_3rd_sec_f[test_tone_minus1]),'b^', label='tone2_3rd')
plt.plot(𝛿f_x_27Hz[test_tone],np.abs(uin_3rd_sec_f[test_tone]),'r^', label='tone3_3rd')
plt.plot(𝛿f_x_27Hz[test_tone_plus1],np.abs(uin_3rd_sec_f[test_tone_plus1]),'g^', label='tone4_3rd')
plt.xlabel('freq (Hz)')
plt.ylabel('Magnitude')
plt.title('|Uin|')
plt.legend()
plt.grid()

plt.subplot(142)  #uin phase plot
plt.scatter(uin_1st_sec_f[test_tone_minus1].real, uin_1st_sec_f[test_tone_minus1].imag,c='b',marker='*',label='tone2_1st')
plt.scatter(uin_1st_sec_f[test_tone].real, uin_1st_sec_f[test_tone].imag,c='r',marker='*',label='tone3_1st')
plt.scatter(uin_1st_sec_f[test_tone_plus1].real, uin_1st_sec_f[test_tone_plus1].imag,c='g',marker='*',label='tone4_1st')
plt.scatter(uin_3rd_sec_f[test_tone_minus1].real,uin_3rd_sec_f[test_tone_minus1].imag,c='b',marker='^',label='tone2_3rd')
plt.scatter(uin_3rd_sec_f[test_tone].real,uin_3rd_sec_f[test_tone].imag,c='r',marker='^',label='tone3_3rd')
plt.scatter(uin_3rd_sec_f[test_tone_plus1].real,uin_3rd_sec_f[test_tone_plus1].imag,c='g',marker='^',label='tone4_3rd')
plt.xlabel('real')
plt.ylabel('imag')
plt.title('Uin Phase')
plt.legend()
plt.grid()

plt.subplot(143) #p magnitude plot
plt.plot(𝛿f_x_27Hz[test_tone_minus1],np.abs(p_1st_sec_f[test_tone_minus1]),'b*', label='tone2_1st')
plt.plot(𝛿f_x_27Hz[test_tone],np.abs(p_1st_sec_f[test_tone]),'r*', label='tone3_1st')
plt.plot(𝛿f_x_27Hz[test_tone_plus1],np.abs(p_1st_sec_f[test_tone_plus1]),'g*', label='tone4_1st')
plt.plot(𝛿f_x_27Hz[test_tone_minus1],np.abs(p_3rd_sec_f[test_tone_minus1]),'b^', label='tone2_3rd')
plt.plot(𝛿f_x_27Hz[test_tone],np.abs(p_3rd_sec_f[test_tone]),'r^', label='tone3_3rd')
plt.plot(𝛿f_x_27Hz[test_tone_plus1],np.abs(p_3rd_sec_f[test_tone_plus1]),'g^', label='tone4_3rd')
plt.xlabel('freq (Hz)')
plt.ylabel('Magnitude')
plt.title('|pressure|')
plt.legend()
plt.grid()

plt.subplot(144)  #p phase plot
plt.scatter(p_1st_sec_f[test_tone_minus1].real, p_1st_sec_f[test_tone_minus1].imag,c='b',marker='*',label='tone2_1st')
plt.scatter(p_1st_sec_f[test_tone].real, p_1st_sec_f[test_tone].imag,c='r',marker='*',label='tone3_1st')
plt.scatter(p_1st_sec_f[test_tone_plus1].real, p_1st_sec_f[test_tone_plus1].imag,c='g',marker='*',label='tone4_1st')
plt.scatter(p_3rd_sec_f[test_tone_minus1].real, p_3rd_sec_f[test_tone_minus1].imag,c='b',marker='^',label='tone2_3rd')
plt.scatter(p_3rd_sec_f[test_tone].real, p_3rd_sec_f[test_tone].imag,c='r',marker='^',label='tone3_3rd')
plt.scatter(p_3rd_sec_f[test_tone_plus1].real, p_3rd_sec_f[test_tone_plus1].imag,c='g',marker='^',label='tone4_3rd')
plt.xlabel('real')
plt.ylabel('imag')
plt.title('pressure phase')
plt.legend()
plt.grid()

These are very important features, closed-loop PID systems. The PID params basically translated into loop filter and loop gain.
- Information about closed-loop filter(H_LF), H_LF bandwidth
- If H_LF bandwidth is too width, the tone9 will wander back and forth in a wide range
- If H_LF bandwidth is too narrow, the system may not reach a steady state, or if its does, very jittery in tone3
- Tone3 pressure's phase delay in 1st sec is always 180 degree phase shifted wrt 3rd sec.
- In communication systems, they use this phase error to lock the receiver clock with its transmitter clock.

LUNG Transfer Function
- If the freq responses of u_in(f) and pressure(f) are known
- err(f) = pressure(f) - LUNG_f .* u_in(f)
- LUNG_f = pressure(f) .* conj(u_in) ./ ( u_in .* conj(u_in) ), Lung freq response, if err(f)=0.
- LUNG_t = ifft( LUNG_F ), Lung impulse response
- LUNG_z = Z-transform of LUNG_f

In [ ]:
#should not include end point for reconstruction
N=4  #linespace endpoint 0.5 is include by default
np.arange(N), np.linspace(-0.5,0.5,N), np.linspace(-.5, .5, N, endpoint=False), (np.arange(N)-N//2)/N

In [ ]:
#https://stackoverflow.com/questions/67350588/example-python-nfft-fourier-transform-issues-with-signal-reconstruction-normal

def sine_lung():
    # number of sample points
    N = 400 #400
    f1=5   #50.0  #Hz
    f2=20  #80.0 #Hz
    
    f3=10
    f4=0
    
    x_t = np.linspace(0.0,0.5-0.02, N) + np.random.random((N)) * 0.001  #non-uniform
    #y = np.sin( 2.0*np.pi *f1 *x_t) + 0.5 * np.sin( 2.0* np.pi* f2* x_t)
    p = np.sin( 2.0*np.pi *f1 *x_t) + 0.5 * np.sin( 2.0* np.pi* f2* x_t + np.pi/2)
    u = np.sin( 2.0*np.pi *f3 *x_t) + 0.5 * np.sin( 2.0* np.pi* f4* x_t)

    p_f = nfft(x_t, p)
    u_f = nfft(x_t, u)    
    x_f = np.fft.fftfreq(N,1./N)
    
    plt.figure(figsize=(22,4))
    plt.subplot(131)
    plt.plot(x_t,p,'r')
    plt.xlabel('time-samples')
    plt.legend(['pressure 5Hz,20Hz'])
    plt.grid()  
    
    plt.subplot(132)
    plt.plot(x_f[:int(N/2)], np.abs(p_f[:int(N/2)]), color='red')
    plt.legend(['pressure 5Hz,20Hz'])
    plt.xlabel('freq (Hz)')
    plt.grid()
    
    N2 = N  
    p_fft_k = nfft_adjoint(x_t, p, N2, truncated=False)   #fft output  -ve freq, dc, +ve freq
    u_fft_k = nfft_adjoint(x_t, u, N2, truncated=False)   #fft output  -ve freq, dc, +ve freq
    #(f_k.r[-N2],f_k.i[-N2]), (f_k.r[- (N2-1)],f_k.i[-(N2-1)]),... 
    #(f_k.r[-1],f_k.i[-1]),(f_k.r[0],f_k.i[0]),(f_k.r[1],f_k.i[1])..,(f_k.r[N2-1],f_k.i[N2-1])
    
    #Reconstruct the original signal with nfft
    p_recon = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(p_fft_k))) / (2*N / N2)
    #x_recon = np.linspace(-.5, .5, N2, endpoint=False)
    x_recon = (np.arange(N2)-N2//2)/N2  #should not include end point
    
    #Plot original vs reconstructed
    plt.subplot(133)
    plt.title('nfft')
    plt.plot(x_t, p, 'b', label='p(x)') #original time-domain real data
    plt.plot(np.flipud(x_recon), p_recon.real, 'r--', label='p_recon(x)')  #reconstructed time-domain real-part, residual imag-part
    plt.xlabel('time-samples')
    plt.legend(['orig p','recon p'])
    plt.grid()
    print(np.sum(abs(p_recon.imag)))
    

    #######################################################################
    # comparing u_f, p_f, p_k  (all freq-domain)
    #######################################################################
    plt.figure(figsize=(22,4))
    plt.subplot(131)
    plt.plot(np.abs(u_f))
    plt.plot(np.abs(p_f))
    plt.plot(np.abs(p_fft_k))
    plt.legend(['u','p','p_fft_k'])
    plt.grid()
    
    #ifftshift
    plt.subplot(132)
    plt.plot(np.abs(p_fft_k))          #fft output=  -ve freq, dc, +ve freq
    plt.plot(np.abs(np.fft.ifftshift(p_fft_k)),'r') # dc,+ve freq,-ve freq
    plt.legend(['p_fft_k','ifftshift(p_fft_k)'])
    plt.grid()
    
    plt.subplot(133)
    plt.plot(x_t,p+4)  #original,  +/- 4 for display
    t1 = np.fft.ifft(np.fft.ifftshift(p_fft_k)/ (2*N / N2)).real  #shift freq-domain signal, then ifft
    plt.plot(x_recon,t1,'r')    
    t2=np.fft.fftshift(t1)                        #shift time-domain signal
    plt.plot(x_recon,t2-4,'g')  #-4 for display
    plt.plot(x_recon,np.flipud(t2)-4,'c')         #flipud to recon
    plt.legend(['p','t1=ifft(ifftshift(p_fft_k))','ifftshift(t1)','recon p',])
    plt.grid()
    
    #using irfft to reconstruct
    f_pos_freq = np.fft.ifftshift(p_fft_k)/ (2*N / N2)  
    f_pos_freq = f_pos_freq[:N//2 +1]  #200
    #pinv     = irfft(f_pos_freq, n=len(p))
    pinv     = irfft(f_pos_freq)
    pinv2    = np.fft.fftshift(pinv) 
    p_rec    = np.flipud(pinv2)
    p_recon  = p_rec[N//2:]
    
    plt.figure(figsize=(22,4))
    plt.subplot(131)
    plt.plot(np.abs(f_pos_freq))
    plt.legend('Pressure +ve freq')
    plt.xlabel('freq (Hz)'),plt.grid()
    
    plt.subplot(132)
    plt.plot(pinv+2)  #2 for display
    plt.plot(pinv2)
    plt.plot(p_rec)
    plt.legend(['irfft(+ve freq)','fftshift(p)','p_rec'])
    plt.xlabel('time-domain'),plt.grid()
    
    #x_recon1 = np.linspace(0, .5, N2//2, endpoint=False)
    plt.subplot(133)
    plt.plot(x_t,p, 'b', label='p(x)') #original time-domain real data
    plt.plot(x_recon[x_recon>=0],p_recon,'r',label='p_recon')
    plt.legend()
    plt.xlabel('time-domain'),plt.grid()
    
    print(f'len=',np.shape(pinv),',DC=',pinv[0],',Nyq=',pinv[-1],np.shape(p_recon))
    
    return p,u, np.fft.ifftshift(u_fft_k)[:int(N2/2)+1], np.fft.ifftshift(p_fft_k)[:int(N2/2)+1], x_t, x_f

In [ ]:
def reconstruct(ut1, p_norm1, uin_norm1, p_norm1_27Hz, uin_norm1_27Hz):      
    #From the last example above
    #native_fs = np.ceil(1/dt_mean)   #native fs
    #uf = np.abs(nfft(ut, uin_norm))  #ut,native time-step, non-uniform
    #pf = np.abs(nfft(ut, p_norm))
    #uin_norm_27Hz_f = nfft(𝛿t_axis_27Hz, uin_norm_27Hz) 
    #p_norm_27Hz_f   = nfft(𝛿t_axis_27Hz, p_norm_27Hz)
    #xf = np.fft.fftfreq(N,1./N)  #[0,1,.....(N-1),-N,-(N-1),..-2,-1], index 0 is DC, index N/2 is Nyq
    #xf = np.fft.fftfreq(N,1./N)  #[0.,1...38.,39.,-40.,-39.,..-2,-1], freq bin in index
    #xf = 𝛿f_axis_27Hz            #[0.,0.33333333,0.66666667,1.,..,26., 26.33333333], freq bin in Hz
    #f_res = 1/dt_mean/N
    
    #After FFT, all time-steps are in uniform, use uniform ifft to reconstruct, then interpolate back to native fs
    #y -> iFFT( FFT(y)) -> y_hat(uniform), interp1d_reverse(ut, y_hat)-> y_hat_native (native time-step)

    N=len(ut1)   #native time step
    N2=N
    #x = np.linspace(0.0,0.5, N) 

    ut1_pt5 = np.zeros(N)
    𝛿t_27Hz_pt5 = np.zeros(N)
    for i in range(N):
        ut1_pt5[i] = 0.5*ut1[i]/ut1[-1]
        𝛿t_27Hz_pt5[i] = 0.5*𝛿t_axis_27Hz[i]/𝛿t_axis_27Hz[-1]
        
    #f_p_norm_k = nfft_adjoint(x, p_norm1, N2, truncated=False) 
    #f_p_norm_27Hz_k = nfft_adjoint(x, p_norm1_27Hz, N2, truncated=False) 

    f_uin_norm_k      = nfft_adjoint(ut1_pt5, uin_norm1, N2, truncated=False) 
    f_uin_norm_27Hz_k = nfft_adjoint(𝛿t_27Hz_pt5, uin_norm1_27Hz, N2, truncated=False) 
    f_p_k             = nfft_adjoint(ut1_pt5, p_norm1, N2, truncated=False) 
    f_p_27Hz_k        = nfft_adjoint(𝛿t_27Hz_pt5, p_norm1_27Hz, N2, truncated=False) 
    #f_px_k = nfft_adjoint(x, p_norm1, N2, truncated=False) #
    #y_px_recon = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(f_px_k))) / (2*N / N2)
    
    y_p_recon      = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(f_p_k))) / (2*N / N2)
    x_recon_native = 2*ut1[-1]*np.linspace(-.5, .5, N2, endpoint=False)   #maps -3sec to +3sec 
    y_p_27Hz_recon = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(f_p_27Hz_k))) / (2*N / N2)
    x_recon_27Hz   = 2*𝛿t_axis_27Hz[-1]*np.linspace(-.5, .5, N2, endpoint=False)   #maps -3sec to +3sec 
    #print(x[-1],ut1[-1])
    
    #Reconstruct the uniform signal with ifft, because after nfft, time-step is in uniform
    y_p_norm_recon        = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(f_p_k))) / (2*N / N2)
    y_p_norm_27Hz_recon   = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(f_p_27Hz_k))) / (2*N / N2)
    y_uin_norm_recon      = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(f_uin_norm_k))) / (2*N / N2)
    y_uin_norm_27Hz_recon = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(f_uin_norm_27Hz_k))) / (2*N / N2)
    #x_recon = 6*np.linspace(-.5, .5, N2, endpoint=False)   #maps -3sec to +3sec 

    p_nat      = interp1d_reverse(x_recon_native[x_recon_native>=0], np.flipud(x_recon_native)[:int(N/2)], y_p_norm_recon[:int(N/2)].real )  
    p_27Hz_nat = interp1d_reverse(x_recon_native[x_recon_native>=0], np.flipud(x_recon_27Hz)[:int(N/2)], y_p_norm_27Hz_recon[:int(N/2)].real )
    u_nat      = interp1d_reverse(x_recon_native[x_recon_native>=0], np.flipud(x_recon_native)[:int(N/2)], y_uin_norm_recon[:int(N/2)].real )
    u_27Hz_nat = interp1d_reverse(x_recon_native[x_recon_native>=0], np.flipud(x_recon_27Hz)[:int(N/2)], y_uin_norm_27Hz_recon[:int(N/2)].real )
    
    #time-domain
    plt.figure(figsize=(22,4))
    plt.subplot(131)
    plt.plot(ut1, p_norm1, 'b',label='p_norm') #original time-domain real data
    plt.plot(𝛿t_axis_27Hz, p_norm1_27Hz,'b--', label='p_norm_27Hz')

    plt.plot(ut1, uin_norm1, 'g',label='uin_norm') #original time-domain real data
    plt.plot(𝛿t_axis_27Hz, uin_norm1_27Hz,'g--', label='uin_norm_27Hz')
    plt.xlabel('time-samples (sec)')
    plt.legend(), plt.grid()
    
    #freq-domain
    plt.subplot(132)
    plt.plot(2*native_fs*(ut1_pt5)[:int(N/2)+1], np.abs(np.fft.ifftshift(f_p_k)[:int(N/2)+1]),'b',label='p_norm') 
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(np.fft.ifftshift(f_p_27Hz_k)[:int(N/2)+1]), 'b--', label='p_norm_27Hz')  

    plt.plot(2*native_fs*(ut1_pt5)[:int(N/2)+1], np.abs(np.fft.ifftshift(f_uin_norm_k)[:int(N/2)+1]),'g',label='uin_norm') 
    plt.plot(54*(𝛿t_27Hz_pt5)[:int(N/2)+1], np.abs(np.fft.ifftshift(f_uin_norm_27Hz_k)[:int(N/2)+1]), 'g--', label='uin_norm_27Hz')    
    plt.xlabel('freq (Hz)')
    plt.legend(),plt.grid()

    #time-domain reconstruct to uniform time step
    plt.subplot(133)
    plt.title('nfft')
    plt.plot(ut1, p_norm1, 'b',label='p_norm') #original time-domain real data
    plt.plot(np.flipud(x_recon_native), y_p_norm_recon.real, 'b+', label='p_norm_recon')    
    plt.plot(np.flipud(x_recon_27Hz), y_p_norm_27Hz_recon.real, 'b--', label='p_norm_27Hz_recon')     
    print(np.sum(abs(y_p_norm_27Hz_recon.imag)))    
   
    plt.plot(ut1, uin_norm1, 'g',label='uin_norm') #original time-domain real data
    plt.plot(np.flipud(x_recon_native), y_uin_norm_recon.real, 'g+', label='uin_norm_recon') 
    plt.plot(np.flipud(x_recon_27Hz), y_uin_norm_27Hz_recon.real, 'g--', label='uin_norm_27Hz_recon') 
    plt.xlabel('time-samples (sec)')
    plt.legend(), plt.grid()     
    
    #time-domain reconstruct back to native time step
    plt.figure(figsize=(22,4))
    plt.subplot(121)
    plt.plot(ut1, p_norm1, 'b',label='p_norm') #original time-domain real data    
    plt.plot(x_recon_native[x_recon_native>=0], p_nat, 'c',label='p_nat')
    plt.plot(x_recon_native[x_recon_native>=0], p_27Hz_nat, 'k',label='p_27Hz_nat')
    plt.xlabel('time-samples (sec)')
    plt.legend(), plt.grid() 
    
    plt.subplot(122)
    plt.plot(ut1, uin_norm1, 'b',label='uin_norm') #original time-domain real data  
    plt.plot(x_recon_native[x_recon_native>=0], u_nat, 'c',label='uin_nat')
    plt.plot(x_recon_native[x_recon_native>=0], u_27Hz_nat, 'k',label='uin_27Hz_nat')
    plt.xlabel('time-samples (sec)')
    plt.legend(), plt.grid()
    
    return

In [ ]:
#Example using two non-uniform sine tones 5Hz and 20Hz to reconstruct.
p_s,u_s,u_sine_pos, p_sine_pos, t_axis_sine, f_axis_sine = sine_lung()

First row plot:
- First plot is non-uniform, two sine waves(5Hz and 20Hz) in time-domain
- Middle plot is non-uniform nFFT in freq-domain
- Last plot is reconstructed by iFFT and compared with the first plot.

Second row plot:
- First plot, comparing nFFT, nFFT_adjoint. Also showing Pressure(5Hz,20Hz), Uin(10Hz)
- Middle plot, the blue is nfft_adjoint format(-ve freq, dc, +ve freq). To take ifft you need format in red (dc,+ve freq,-ve freq) by performing ifftshift in freq
- Last plot, to show iffshift in time, and flipud

Third row plot:
- First plot, Pressure in freq-domain, 
- Middle plot, using irfft to reconstruct (dc,pos freq,Nyq). Only use pos freq to work on the Lung transfer function. Neg freq can be obtained from pos freq.
- Last plot, reconstructed time-domain signal

In [ ]:
def lung_tranfer_function(Uin_r, Prs_r, xf1, t_axis):
    #Uin_r  => 41, dc,pos_freq,Nyq 
    #Prs_r  => 41, dc,pos_freq,Nyq 
    #xf1 => 41, freq(0,Nyq)
    N = len(t_axis)  #80
    N2 = N
    
    Uin = Uin_r/2.
    Prs = Prs_r/2.
    Iden=np.eye(len(Prs))
    #In Freq-domain
    #err = Prs - L_out,  L_out = L .* Uin
    #0   = Prs - L .* Uin, for err = 0
    #L   = Prs .* conj(Uin) ./ ( Uin .* conj(Uin) )
    D = Uin.dot(np.conj(np.diag(Uin)))+1  #avoid divide zero
    
    #Lung Transfer function or freq response
    Lung_f = np.dot(np.conj(Uin), np.diag(1./D))*Iden  #  np.conj(Uin) ./ ( Uin .* conj(Uin) )
    Lung_f = np.dot(Prs,Lung_f)
    print(np.shape(Uin),np.shape(Prs),np.shape(Lung_f))    

    x_Lung = np.linspace(0, .5, N2, endpoint=True)
    #x_recon1 = np.linspace(0, .5, N2//2, endpoint=False)

    #ifft Lung_out
    l_out_2x_fft = np.zeros(2*len(Lung_f)-1, dtype=complex)       #80 
    l_out_2x_fft[:len(Lung_f)] = Lung_f

    pinv     = irfft(l_out_2x_fft)         #time-domain
    pinv2    = np.fft.fftshift(pinv)       #time-swap
    y_Lung_f = np.flipud(pinv2)            #160
    y_Lung   = y_Lung_f[2*N//2:]           #80
   
    plt.figure(figsize=(22,4))
    plt.subplot(141)
    plt.plot(xf1, np.abs(Prs), 'b')
    plt.plot(xf1, np.abs(Uin), 'r--')
    plt.legend(['pressure','u_in'])
    plt.xlabel('freq (Hz)')
    plt.grid()

    plt.subplot(142)    
    plt.plot(xf1, np.abs(Lung_f), color='b')
    plt.legend(['Lung_f'])
    plt.xlabel('freq (Hz)')
    plt.grid()

    #Plot original vs reconstructed
    plt.subplot(143)
    plt.title('ifft')
    plt.plot( y_Lung, 'b', label='Lung_t')  #reconstructed time-domain real-part, residual imag-part
    plt.xlabel('time-samples')
    plt.legend()
    plt.grid()
    print(np.sum(abs(y_Lung.imag)))
    
    #plt.figure()
    #plt.plot( np.abs(Lung_Nyq), color='b')
    
    return xf1, Lung_f, x_Lung, y_Lung

In [ ]:
#Example using two non-uniform sine tones 5Hz and 20Hz as Pressure, non-uniform sine tone 5Hz as u_in.
# Err = Pressure - Lung .* u_in
# if Err = 0, solve for Lung Freq/Impulse responses. Basically it's a freq-domain equalizer(FEQ), equalize the phase and amplitude.
f_axis_lung,lung_f, t_axis_lung,lung_t = lung_tranfer_function(u_sine_pos,
                                                                p_sine_pos,
                                                                800*t_axis_sine[:int(len(t_axis_sine)/2)+1],  #double ifft points
                                                                t_axis_sine
                                                               )

#np.shape(f_axis_lung), np.shape(lung_f), np.shape(t_axis_lung), np.shape(lung_t)

Lung Transfer Function
- Left plot: Pressure (5Hz and 20 Hz), Uin (10Hz)
- Middle plot: Lung_f magnitude response. It blocked Uin tone 10Hz, and output tones 5Hz and 20Hz to match the target. 
- Last plot: Lung_t model in time-domain.

Lung Output
- Freq-domain Method, Lung_out
- Time-domain Method, Lung_out
- Finding Delay between Target and Lung_out, this delay is Breath boundary
- Assuming breath is repetitive/periodic
- Adjusting delay Lung_out in time-domain to match its Target

In [ ]:
u_sine_dc = u_s.mean()
p_sine_dc = p_s.mean()
l_gain_sine = u_sine_dc/p_sine_dc
print(f'u_dc=',u_sine_dc,',p_dc=',p_sine_dc)
print('lung DC gain =',l_gain_sine)

In [ ]:
#######################################################################
# Freq-domain Method:
# Lung_output is freq multiply to match its Target Pressure
# Lung_out_f = Lung_f * U_F
# Lung_out_t = ifft(Lung_out_f)
####################################################################### 
len_p = len(t_axis_sine)                #400
U_pos = u_sine_pos/2.  #201,   np.fft.ifftshift(u_sine_fft)/ 2  #400, get U positive freq
P_pos = p_sine_pos/2.  #201,   np.fft.ifftshift(p_sine_fft)/ 2  #400, get Pressure positive freq
#U_pos = U_F[:len_p//2 +1]  #201
#P_pos = P_F[:len_p//2 +1]  #201

l_out_fft = lung_f * U_pos

plt.figure(figsize=(22,4))
plt.subplot(131)
plt.plot(np.abs(P_pos),'b')
plt.plot(np.abs(l_out_fft),'r--')
#plt.plot(np.abs(lung_f),'g--')
plt.legend(['Pressure','Lung_output'])
plt.xlabel('freq-domain')
plt.grid()

#ifft Lung_out from freq-domain
l_out_2x_fft = np.zeros(2*len(l_out_fft)-1, dtype=complex)       #401, double ifft point
l_out_2x_fft[:len(l_out_fft)] = l_out_fft
#pinv     = irfft(l_out_fft)          #time-domain
pinv     = irfft(l_out_2x_fft)        #time-domain
pinv2    = np.fft.fftshift(pinv)      #time-swap
y_Lung_f = np.flipud(pinv2)           #800
y_Lung   = y_Lung_f[2*len_p//2:]      #401
y_Lung   = 2*y_Lung

plt.subplot(132)
plt.plot(p_s)
plt.plot(y_Lung,'r--')
#plt.plot(y_Lung/l_gain_sine,'r--')
plt.legend(['Pressure','Lung_output'])
plt.xlabel('time-domain')
plt.grid()

#x_recon = np.linspace(0, .5, N1//2, endpoint=False)
#x_recon = (np.arange(len_p)-len_p//2)/len_p
x_recon = np.linspace(0, .5, len_p, endpoint=False)   #maps -3sec to +3sec 

#Interpolate back to native fs, non-uniform time-step
p_recon_native = interp1d_reverse(t_axis_sine, x_recon, y_Lung)  #

#Undo norm, std(p) then add p_dc
#p_recon_norm_native = np.std(p_s)*p_recon_native+p_sine_dc
p_recon_norm_native = p_recon_native

#plt.subplot(132)
#plt.plot(pinv+2)  #2 for display
#plt.plot(pinv2)
#plt.plot(p_rec)
#plt.legend(['irfft(+ve freq)','fftshift(p)','p_rec'])
#plt.xlabel('time-domain'),plt.grid()
    
plt.subplot(133)
plt.plot(t_axis_sine, p_s, 'b', label='p(x)') #original time-domain real data
plt.plot(t_axis_sine, p_recon_norm_native,'r',label='p_recon_native')
plt.legend()
plt.xlabel('time-domain (sec)'),plt.grid()  

#plt.figure()
#plt.plot(2*(t_axis_sine[len(t_axis_sine)//2:]-.25), p_s[len(t_axis_sine)//2:], 'b', label='p(x)') #original time-domain real data
#plt.plot(x_recon_native[x_recon_native>=0],2*p_recon_native1,'r',label='p_recon_native')
#plt.plot(t_axis_sine, p_s, 'b', label='p(x)') #original time-domain real data
#plt.plot(t_axis_sine, p_recon_native,'r',label='p_recon_native')

#######################################################################
# Time-domain Method: 
# Lung_output by time-domain conv to match its Target Pressure
# Lung_out_t = conv(lung_t, uin)
#
# find delay in time-domain
# correlate(uniform target, non uniform Lung_out_t)
#######################################################################   
l_out_s = np.convolve(lung_t, u_s)   #Lung output in time-domain = conv(lung_t, uin), len(lung_t)+len(u)-1  samples
corr_out_s = np.correlate(p_s,l_out_s,'full')   #corr(non uniform target, uniform Lung output), len(lung_t)+len(p)-1  samples
#corr_out = np.correlate(l_out,u,'full')  #corr(Lung output, uin)

#Find Delay/breath boundary
peak_loc_s = np.argmax(np.abs(corr_out_s))
delay_bdry_s = len(corr_out_s)-peak_loc_s
delay_s=0
if delay_bdry_s <= len_p:
    delay_s = len_p-delay_bdry_s
    l_out1_s = np.roll(l_out_s,delay_s)[:len_p]
print(f'delay =',delay_s,'samples,  delay_bdry_s=',delay_bdry_s,'samples, peak_loc=',peak_loc_s, ', max_peak=', np.max(np.abs(corr_out_s)))  
    
#Interpolate back to native fs, non-uniform time-step
p_recon_t_native = interp1d_reverse(t_axis_sine, x_recon, l_out1_s )  # 
#Undo norm, std(p) then add p_dc
#p_recon_t_native = 2*np.std(p_s)*p_recon_t_native+p_s_dc    
    
plt.figure(figsize=(22,4))
plt.subplot(141)
plt.plot(p_s)
plt.legend('p')
plt.xlabel('time-domain')
plt.grid()

plt.subplot(142)
plt.plot(5*lung_t)  #display
plt.plot(u_s)
plt.legend(['lung_t','uin'])
plt.xlabel('time-domain')
plt.grid()

plt.subplot(143)
plt.plot(corr_out_s)
plt.legend(['corr_out'])
plt.xlabel('time-domain')
plt.grid()

plt.subplot(144)
plt.plot(p_s)
plt.plot(l_out_s)
plt.plot(l_out1_s,'r--')
plt.xlabel('time-domain')
plt.grid()
plt.legend(['p','l_out','l_out adjust delay'])

plt.figure()
plt.plot(t_axis_sine, p_s)
plt.plot(t_axis_sine, p_recon_t_native,'r--')
plt.legend(['p','l_out_nat'])
plt.xlabel('time-domain(sec)')
plt.grid()

First row plot: Freq-domain Method
- Freq-domain Lung output, it matched the Target
- iFFT(Freq-domain Lung output)
- Time-domain, transform back to native time step

Second row plot: Time-domain Method
- First plot, Target/Pressure in time-domain
- Second plot, Time-domain Lung_t, Lung input(Uin)
- Third plot, correlate(Pressure, Lung_out), then find peak location as delay
- Last plot, Lung_out and Pressure. (work in progress)

Third row plot:
- Pressure vs. Lung_out in native fs from time-domain method (work in progress)


In [ ]:
N = len(ut)
#x = np.linspace(0.0,0.5, N)
ut_pt5 = np.zeros(N)
𝛿t_27Hz_pt5 = np.zeros(N)
for i in range(N):
    ut_pt5[i] = 0.5*ut[i]/ut[-1]
    𝛿t_27Hz_pt5[i] = 0.5*𝛿t_axis_27Hz[i]/𝛿t_axis_27Hz[-1]    #norm to 0.5
    #print(ut1_pt5[i])

In [ ]:
#Reconstruct u_in, pressure, with native fs vs interpolated fs=27KHz
reconstruct(ut,p_norm,uin_norm,         #ut=native time-step, p_norm=pressure, uin_norm=uin
            p_norm_27Hz,uin_norm_27Hz,  #p_norm_27Hz=interp(p_norm), uin_norm_27Hz=interp(uin_norm)
            )

Reconstruction Pressure and Uin (non-uniform vs uniform)

Top row plot:
- dash plot is uniform, solid plot is non-uniform

Bottom row plot:
- This is the last step, transforming uniform signal back to the native time step.
- For the interpolation case, due to interpolation having a "natural" smoothing action and tend to represent lower freq, they cannot reconstruct the spiky saw-tooth wave.
- The interpolation can be improved by upsample many times then interpolate.
- For native fs case, somehow it has smoothing action too.

In [ ]:
#Lung Tranfer Function based on non-uniform native fs (u_in, pressure)
f_axis_27Hz,lung_f_27Hz, t_axis_27Hz, lung_t_27Hz = lung_tranfer_function(uin_norm_27Hz_f,    #41, dc,pos_freq,Nyq 
                                                                          p_norm_27Hz_f,      #41, dc,pos_freq,Nyq  
                                                                          54*(𝛿t_27Hz_pt5)[:int(N/2)+1],  #41 freq(Hz),dc,pos_freq,Nyq 
                                                                          𝛿t_27Hz_pt5)        #N=80, dc,pos_freq,Nyq,neg_freq

Lung Transfer Function (uniform fs=27KHz)
- The magnitudes of Lung_f and amplitude Lung_t are small because pressure and u_in are similar

Lung DC gain

In [ ]:
l_gain = Uin_interp_fltr_27Hz_dc/ P_interp_fltr_27Hz_dc
l_gain_ideal = uin_dc/ p_dc
p_dc_err = p_dc-P_interp_fltr_27Hz_dc
u_dc_err = uin_dc-Uin_interp_fltr_27Hz_dc
print(f'u_dc=',uin_dc,',p_dc=',p_dc,',u_intrp_dc=',Uin_interp_fltr_27Hz_dc,',p_intrp_dc=',P_interp_fltr_27Hz_dc)
print('lung DC gain =',l_gain, ', P DC intrp err=',p_dc_err, ', U DC intrp err=',u_dc_err)

Lung Output
- Freq-domain Method
- Time-domain Method

In [ ]:
#######################################################################
# Freq-domain Method:
# Lung_output is freq multiply to match its Target Pressure
# Lung_out_f = Lung_f * U_F
# Lung_out_t = ifft(Lung_out_f)
####################################################################### 
len_p = len(𝛿t_27Hz_pt5)  #80

U_pos = uin_norm_27Hz_f/2.  #41, dc,pos_freq,Nyq  
P_pos = p_norm_27Hz_f/2.    #41, dc,pos_freq,Nyq  
l_out_fft = lung_f_27Hz * U_pos

plt.figure(figsize=(22,4))
plt.subplot(131)
plt.plot(np.abs(P_pos),'b')
plt.plot(np.abs(l_out_fft),'r--')
plt.legend(['Pressure','Lung_output'])
plt.xlabel('freq')
plt.grid()    

#ifft Lung_out_t
l_out_2x_fft = np.zeros(2*len(l_out_fft)-1, dtype=complex)       #2(41)-1 = 81, double ifft points 
l_out_2x_fft[:len(l_out_fft)] = l_out_fft
#pinv     = irfft(l_out_fft)           #time-domain
pinv     = irfft(l_out_2x_fft)         #time-domain
pinv2    = np.fft.fftshift(pinv)       #time-swap
y_Lung_f = np.flipud(pinv2)            #160
y_Lung   = y_Lung_f[2*len_p//2:]       #80
y_Lung   = 2*y_Lung

plt.subplot(132)
plt.plot(p_norm_27Hz)
plt.plot(y_Lung,'r--')
#plt.plot(y_Lung+l_gain_ideal,'r--')
plt.legend(['Pressure','Lung_output'])
plt.xlabel('time-domain')
plt.grid()

#Interpolate back to native fs, non-uniform time-step
#p_recon_norm_native = interp1d_reverse(ut, (27/4.)*𝛿t_27Hz_pt5, y_Lung/l_gain )  #27/2 * 1/2 (double ifft pt)
p_recon_norm_native = interp1d_reverse(ut, (27/4.)*𝛿t_27Hz_pt5, y_Lung )  #27/2 * 1/2 (double ifft pt)

#Undo norm, std(p) then add p_dc
p_recon_norm_native = np.std(p)*p_recon_norm_native+p_dc

plt.subplot(133)
plt.plot(ut, p, 'b', label='p(x)') #original time-domain real data
plt.plot(ut, p_recon_norm_native,'r',label='p_recon_native')
plt.legend()
plt.xlabel('time-domain (sec)'),plt.grid()     

#######################################################################
# Time-domain Method: 
# Lung_output by time-domain conv to match its Target Pressure
# Lung_out_t = conv(lung_t, uin)
#
# find delay in time-domain
# correlate(uniform target, uniform Lung_out_t)
#######################################################################    
l_out = np.convolve(lung_t_27Hz, uin_norm_27Hz)   #Lung output in time-domain = conv(lung_t, uin), 80+40-1 = 119 samples
corr_out = np.correlate(p_norm_27Hz,l_out,'full')  #corr(target,Lung output) , 119+80-1 = 198 samples
peak_loc = np.argmax(np.abs(corr_out))
delay_bdry = len(corr_out)-peak_loc
delay=0
if delay_bdry <= len_p:
    delay = len_p-delay_bdry
    l_out1 = np.roll(l_out,delay)[:len_p]
print(f'delay=',delay,'samples, delay_bdry=',delay_bdry,'samples, peak_loc=',peak_loc, ',max_peak=', np.max(np.abs(corr_out)))   


#Interpolate back to native fs, non-uniform time-step
#p_recon_norm_t_native = interp1d_reverse(ut, (27/4.)*𝛿t_27Hz_pt5, l_out1/l_gain )  #27/2 * 1/2 (double ifft pt)
p_recon_norm_t_native = interp1d_reverse(ut, (27/4.)*𝛿t_27Hz_pt5, l_out1 )  #27/2 * 1/2 (double ifft pt)
#Undo norm, std(p) then add p_dc
p_recon_norm_t_native = np.std(p)*p_recon_norm_t_native+p_dc

plt.figure(figsize=(22,4))
plt.subplot(141)
plt.plot(p_norm)
plt.plot(p_norm_27Hz)
plt.legend(['p_norm','p_norm_27Hz'])
plt.xlabel('time-domain')
plt.grid()

plt.subplot(142)
plt.plot(5*lung_t_27Hz)  #display
plt.plot(uin_norm_27Hz)
plt.legend(['lung_t_27Hz','uin_norm_27Hz'])
plt.xlabel('time-domain')
plt.grid()

plt.subplot(143)
plt.plot(corr_out)
plt.legend(['corr_out'])
plt.xlabel('time-domain')
plt.grid()

plt.subplot(144)
plt.plot(p_norm_27Hz)
plt.plot(l_out)
plt.plot(l_out1,'g--')
plt.legend(['p_norm_27Hz','l_out','l_out adjust delay'])
plt.xlabel('time-domain')
plt.grid()

plt.figure()
plt.plot(ut, p)
plt.plot(ut, p_recon_norm_t_native,'g--')
plt.legend(['p','l_out_nat'])
plt.xlabel('time-domain (sec)')
plt.grid()

First row plot: Freq-domain Method
- First plot, Freq-domain Lung output, it matched its Target
- Middel plot, iFFT(Freq-domain Lung output)
- Last plot, Transform back to native time steps

Second row plot: Time-domain Method
- First plot, Pressure in time-domain
- Second plot, Time-domain Lung_t, Lung input(Uin)
- Third plot, correlate(Pressure, Lung_out), then find peak location as delay
- Last plot, Lung_out after adjusting delay and Pressure.

Third row plot: 
- Transform back to native time steps from time-domain method

Numerical Values Check for Lung Output(tone9) in Freq-domain

In [ ]:
p9=p_norm_27Hz_f[tone9_idx]           #pressure
u9=uin_norm_27Hz_f[tone9_idx]         #uin 
l9=p9*np.conj(u9)/(u9*np.conj(u9))    #lung

p9_hat = l9*u9                        #lung output
#p9,l9,u9,p9_hat
print(f'pressure=',p9,',Lung out=',p9_hat)

Frequency Error Analysis
- Err = Target - LUNG_predict  =>  E(Z) = R(Z) - Y(Z)
- E(Z)/R(Z) = [1- Y(z)/R(Z)]
- For a unit step reference input, the phase of the input signal has a step change or frequency jumps
- R(Z) = 1/(1- Z^-1), this is unit step reference in Z-domain.
- Applying the final value theorem, lim_(k->inf) e(kT) = lim_(z->1) (1-Z^-1) E(Z) , k= time-index, T= uniform time interval.
- Steady State Error, is related to the proportional Gain, is the DC value evaluated at Z=1, a very important value.


# **Summary**

Analyze Spectral Content of the Signal
- Lomb-Scargle Periodograms
- nFFT, a non-uniform FFT
- Uniform FFT after resample
- Reconstruction of the signal from time to freq then back to time domain

Features Extraction
- Spectral Content
- DC values
- Magnitude, bandwidth, and phase delay

Model of Lung from a Breath of 3sec Instance
- Able to obtain Lung_f(freq) and Lung_t(time) models
- Two methods to generate Target from the Lung Model
- From Freq-domain of Lung output, then ifft(Lung_out in freq-domain)
- From Time-domain of Lung output, this step is much complicated
- Lung_out = convolve( Lung_t, Uin)
- corr_out = correlate( Target, Lung_out)
- Find breath boundary, by finding the peak location of the corr_out
- The peak location is the delay
- Close to match Lung_out with Target in time-domain
- Able to match Lung_out with Target in freq-domain
- During training, the delay is inserted in the Uin, then the during test, there is no need to correlate the lung output with its target to find the delay, NN with automatic generated delay.
- With the Lung Model, you can generate many training data sets
- Cost function should be evaluated at native time-step, not in uniform time-step, then transform to native time-step
- Code need some fine tune
